# Twitter Hashtag Monitoring
#### September 2017 

## Description

- Monitoring a Twitter Hashtag, e.g. #Berlin
- An alert is launched if keywords appear more often than a preset threshold.
- Alert: Email with the text of the tweets containing the keywords is sent
- The program stops as soon as an alert is sent

## Technology used

- Twitter API (Streaming API)
    - Filtering the stream for keywords, e.g. hashtag #Berlin
    - Filtering for language of the tweets: e.g. english and german
- SQLITE Database for storing the tweets containing the keywords
    - Table "TWEETS" with the following columns:
        - ID of the tweet
        - a timestamp
        - Screen name of the user
        - the content of the tweet
- SMTPLIB Package for sending emails
    - sending a text message with the pure text of the tweets for easier understanding
    - Subject: "Berlin Alert"
    - Email can be sent to different recipients

## Limitations

- Each account may create only one standing connection to the public endpoints, and connecting to a public stream more than once with the same account credentials will cause the oldest connection to be disconnected. Clients which make excessive connection attempts (both successful and unsuccessful) run the risk of having their IP automatically banned.
- I had regularily "connection limit exceeded" errors due to frequent testing of the stream and my code. A workaround was it to use a second pair of API keys when the first one was temporarily blocked, until the original stream has been reset by twitter.
- Further limitations apply with receiving a high number of tweets

## Further development

- Shiny Web App providing more detailed and updated information (useful as soon as the alert message is sent)
    - e.g. providing the links which are part of the content of the tweets
    - e.g. providing a current stream of the text of the tweets coming in in realtime
    - e.g. providing a map indicating the user's location posting the tweets
- A more sophisticated approach to the threshold using statistical analysis
    - Necessary: Collecting/analyzing data for a set of keywords over an extended period of time


## Preparations before running the code

- Set your working directory and adapt the link to it in the "Preparing Twitter API" section below
- Create twitter_api_keys.txt with your API keys in your working directory
- Create mailserver.txt with your googlemail address and password in your working directory
- Install Sqlite on your machine and create "monitoring.db" database in your working directory

## How to run the code

- Run the "Importing packages" section
- Run the "Preparing Twitter API" section
- Run the cells of all functions below to source them into memory
- Run the "initiate_sqlite()" function to create a table "TWEETS" in the database
- Run the "start_stream()" function to start the monitoring

### Importing packages

In [ ]:
# Importing this just for fun!
import this
# Import twitter package
import twitter 
# Import json package
import json
# Import sqlite3 package
import sqlite3
# Import smtplib and related packages
import smtplib
import email
from email.mime.text import MIMEText
# Import re package
import re
# Import time package
import time
# Import sys package
import sys

### Preparing Twitter API

In [ ]:
# Setting up working directory
%cd "C:\workingdirectory"
    
# Reading in API Keys
with open("twitter_api_keys.txt", 'r') as i:
    keys = i.read().split()
    
# Reading Keys into Twitter API
api = twitter.Api(consumer_key = keys[0],
                consumer_secret = keys[1],
                access_token_key = keys[2],
                access_token_secret= keys[3])

## SQLITE 

### Function: initiate sqlite3 database 

In [ ]:
def initiate_sqlite():

    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    
    # Adding "TWEETS" table to the database
    conn.execute('''CREATE TABLE TWEETS (ID TEXT PRIMARY KEY,
                 TIME DATE,
                 NAME TEXT, 
                 COORD1 REAL,
                 COORD2 REAL,
                 CONTENT TEXT);''')
    
    conn.commit()
    conn.close()
    
    return

### Function: write to sqlite3 database table

In [ ]:
def sqlite_write(twitter_id, screen_name_of_user, coord1, coord2, text_of_tweet):
    
    # Connecting to "monitoring" database already set up before using sqlite3
    conn = sqlite3.connect("monitoring.db")
    
    # Insert Query "TWEETS" table and excecuting the command
    # using a sqlite timestamp for the time column
    command = "INSERT INTO TWEETS VALUES ('"+twitter_id+"',datetime('now', 'localtime'),'"+screen_name_of_user+"','"+str(coord1)+"','"+str(coord2)+"', '"+text_of_tweet+"');"
    conn.execute(command)
    
    # Commiting and closing the connection to the database
    conn.commit()
    conn.close()
    
    return

### Function: read from sqlite3 database table 

In [ ]:
def sqlite_read():
    
    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    c = conn.cursor()
    
    # Select Query "TWEETS" table for the tweets added to the database in the last 10 minutes
    command = "SELECT * FROM TWEETS WHERE TIME BETWEEN datetime('now', 'localtime', '-10 minutes') AND datetime('now', 'localtime');"
    c.execute(command)
    
    # fetching the selected data into the rows list
    rows = c.fetchall()
    
    # closing the connection to the database
    conn.close()
    
    return rows

### Function: count tweets from the last 10 minutes from sqlite3 database table

In [ ]:
def sqlite_count():
    
    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    c = conn.cursor()
    
    # Select Query "TWEETS" table for the tweets added to the database in the last 10 minutes
    
    command = "SELECT COUNT(*) FROM TWEETS WHERE TIME BETWEEN datetime('now', 'localtime', '-10 minutes') AND datetime('now', 'localtime');"
    c.execute(command)
    
    # fetching the count data
    count = c.fetchall()
    
    # closing the connection to the database
    conn.close()
    
    return count

## SMTPLIB

### Function: sending alert email

In [ ]:
def send_email():

    # Reading in email address and password from text file
    with open("mailserver.txt", "r") as h:
        data = h.read().split() 
        
    mailaddress = data[0]
    mailpassword = data[1]
    
    fromaddr = mailaddress
    toaddrs  = mailaddress

    # Reading in text file with text of tweets containing the keywords
    with open("emailtext.txt", "r") as fp:
        msg = MIMEText(fp.read(), _subtype="plain")
    

    # Preparing email properties like subject etc.
    msg["From"] = fromaddr
    msg["To"] = toaddrs
    msg["Subject"] = "Berlin Alert"
    
    # Sending the email using a Googlemail account
    server = smtplib.SMTP("smtp.googlemail.com", 587)
    server.starttls()
    server.login(mailaddress, mailpassword)
    server.send_message(msg)
    server.quit()
    
    return

## Setting Threshold and launching alert email

### Function: Threshold function

In [ ]:
def threshold():
    
    # Reading in the current count  
    count = sqlite_count()[0][0]
    
    # Preset threshold of tweets within the last 10 minutes
    # here set high to collect and save some tweets into the database before the alert is launched
    if count > 1000:
            
        # reading the tweets from the last 10 minutes from the database    
        results = sqlite_read()
        
        # writing the text of the tweets line by line in a text file
        with open("emailtext.txt", "w") as f:
            for row in results:
                f.write(row[3] + "\n")
                
        # sending email with the text of the tweets
        send_email()
        
        # stopping program when alert has been sent       
        sys.exit()
        
    return

## Stream API

### Function: starting the tweet stream, saving  data to database and running the threshold

In [ ]:
def start_stream():
    
    # Define Hashtag+Keywords and languages for streaming API filter
    
    # e.g. ["#Berlin terror"] would be to filter for "#Berlin" AND "terror"
    # e.g. ["#Berlin terror, #Berlin terroranschlag, #Berlin anschlag"]
    # for easier testing we leave this with "#Berlin" only
    hashtag_to_monitor = ["#Berlin"]
    
    # filtering for english and german language tweets only
    LANGUAGES = ["en", "de"]
    
    # Connecting to stream
    stream = api.GetStreamFilter(track=hashtag_to_monitor, languages=LANGUAGES)        
        
    # Going through the tweets delivered and saving the selected data to the database
    try:
        for tweet in stream:
            
            # make the tweet text suitable for saving in the database and sending by email
            # e.g. removing links, having just text as output for easy readability
            # and understanding. this also helps as a workaround to unsolved problems
            # to send unicode messages by smtplib in python3
            tweet["text"] = re.sub("\'", "", tweet["text"])
            tweet["text"] = re.sub("\"", "\\\"", tweet["text"])
            tweet["text"] = re.sub("\s", " ", tweet["text"])
            tweet["text"] = re.sub("http\S+", "", tweet["text"])
            tweet["text"] = re.sub("\W", " ", tweet["text"])
            
            # calling the sqlite_write function and saving the data to the database
            
            if tweet["coordinates"] != None:
            
                sqlite_write(tweet["id_str"], tweet["user"]["screen_name"], tweet["coordinates"]["coordinates"][0], tweet["coordinates"]["coordinates"][1],tweet["text"])
            else:
                sqlite_write(tweet["id_str"], tweet["user"]["screen_name"], "NULL", "NULL", tweet["text"])
            
            # calling threshold function to check if an alert has to be sent
            threshold()
            
    except Exception as g:
        print("Exception", g)
        
    return

In [ ]:
initiate_sqlite()

In [ ]:
start_stream()